In [1]:
from mainnet_launch.data_fetching.get_state_by_block import (
    get_state_by_one_block,
    build_blocks_to_use,
    get_raw_state_by_blocks,
    safe_normalize_6_with_bool_success,
    safe_normalize_with_bool_success,
)

from mainnet_launch.database.postgres_operations import (
    get_full_table_as_df_with_block,
    get_full_table_as_df,
    get_full_table_as_df_with_tx_hash,
)
from mainnet_launch.database.schema.full import (
    DestinationStates,
    Destinations,
    AutopoolDestinations,
    RebalanceEvents,
    RebalancePlans,
    Blocks,
)
import pandas as pd

from mainnet_launch.constants import AUTO_USD, ETH_CHAIN

destinations = get_full_table_as_df(Destinations, where_clause=Destinations.chain_id == 1)
autopool_destinations = get_full_table_as_df(
    AutopoolDestinations, where_clause=AutopoolDestinations.autopool_vault_address == AUTO_USD.autopool_eth_addr
)
# 2 min to fetch
destination_states = get_full_table_as_df_with_block(
    DestinationStates,
    where_clause=DestinationStates.destination_vault_address.in_(destinations["destination_vault_address"].tolist()),
)
plans = get_full_table_as_df(
    RebalancePlans, where_clause=RebalancePlans.autopool_vault_address == AUTO_USD.autopool_eth_addr
)
events = get_full_table_as_df_with_tx_hash(
    RebalanceEvents, where_clause=RebalanceEvents.autopool_vault_address == AUTO_USD.autopool_eth_addr
)
mainnet_blocks = get_full_table_as_df(Blocks, where_clause=Blocks.chain_id == 1).sort_values("block")

/Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/eth_abi/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [6]:
plans

,file_name,datetime_generated,autopool_vault_address,chain_id,solver_address,rebalance_type,destination_out,token_out,destination_in,token_in,...,in_dest_apr,in_dest_adj_apr,apr_delta,swap_offset_period,num_candidate_destinations,candidate_destinations_rank,projected_swap_cost,projected_net_gain,projected_gross_gain,projected_slippage
0,rebalance_plan_1747876486_0xa7569A44f348d3D70d...,2025-05-22 01:14:46+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x7876F91BB22148345b3De16af9448081E9853830,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,...,0.054361,0.054323,0.054323,14.0,0.0,NaN,48.696771,0.000000,0.000000,0.015000
1,rebalance_plan_1751790625_0xa7569A44f348d3D70d...,2025-07-06 08:30:25+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rebalance_plan_1743982165_0xa7569A44f348d3D70d...,2025-04-06 23:29:25+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rebalance_plan_1744695741_0xa7569A44f348d3D70d...,2025-04-15 05:42:21+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x7583b1589aDD33320366A48A92794D77763FAE9e,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,...,0.073199,0.072660,0.072660,12.0,3.0,0.0,25.234559,138.624499,163.859058,0.032486
4,rebalance_plan_1745052157_0xa7569A44f348d3D70d...,2025-04-19 08:42:37+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xE4545f9dBC30Ccb6Cda6930DDFd69f3D419FcB61,0x5C20B550819128074FD538Edf79791733ccEdd18,...,0.064439,0.064409,0.064409,12.0,3.0,0.0,23.367432,260.629399,283.996831,0.015578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1690,rebalance_plan_1765700143_0xa7569A44f348d3D70d...,2025-12-14 08:15:43+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x366C094C5563CD12AF27b9AfFF2200B0E0D056E0,0x85B2b559bC2D21104C4DEFdd6EFcA8A20343361D,...,0.103896,0.103697,0.103697,15.0,3.0,0.0,6.019116,228.556255,234.575371,0.010329
1691,rebalance_plan_1756880141_0xa7569A44f348d3D70d...,2025-09-03 06:15:41+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x7876F91BB22148345b3De16af9448081E9853830,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,...,0.082677,0.082676,0.082676,15.0,3.0,0.0,3.000000,58.699346,61.699346,0.015000
1692,rebalance_plan_1769062549_0xa7569A44f348d3D70d...,2026-01-22 06:15:49+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x7876F91BB22148345b3De16af9448081E9853830,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,...,0.043441,0.043431,0.043431,15.0,0.0,NaN,49.074345,0.000000,0.000000,0.015000
1693,rebalance_plan_1769055349_0xa7569A44f348d3D70d...,2026-01-22 04:15:49+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x7876F91BB22148345b3De16af9448081E9853830,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,...,0.043361,0.043343,0.043343,15.0,0.0,NaN,98.148689,0.000000,0.000000,0.015000


In [ ]:
limited_events_df = events[
    ["destination_in", "destination_out", "block", "safe_value_out", "rebalance_file_path"]
].copy()

get_fee_and_base_apr = destination_states.set_index(["destination_vault_address", "rebalance_plan_key"])[
    "fee_plus_base_apr"
].to_dict()

limited_events_df["fee_and_base_out"] = limited_events_df.apply(
    lambda row: get_fee_and_base_apr.get((row["destination_out"], row["rebalance_file_path"]), None), axis=1
)
limited_events_df["fee_and_base_in"] = limited_events_df.apply(
    lambda row: get_fee_and_base_apr.get((row["destination_in"], row["rebalance_file_path"]), None), axis=1
)

destination_names = destinations.set_index("destination_vault_address")["underlying_name"].to_dict()
exchange_names = destinations.set_index("destination_vault_address")["exchange_name"].to_dict()
pool_addresses = destinations.set_index("destination_vault_address")["pool"].to_dict()

limited_events_df["destination_in_name"] = limited_events_df["destination_in"].map(destination_names)
limited_events_df["destination_out_name"] = limited_events_df["destination_out"].map(destination_names)
limited_events_df["out_exchange_name"] = limited_events_df["destination_out"].map(exchange_names)
limited_events_df["in_exchange_name"] = limited_events_df["destination_in"].map(exchange_names)
limited_events_df["pool_in"] = limited_events_df["destination_in"].map(pool_addresses)
limited_events_df["pool_out"] = limited_events_df["destination_out"].map(pool_addresses)

# whenever destination out == autoUSD.aupooleeth addr see fee and base out = 0
limited_events_df.loc[limited_events_df["destination_out"] == AUTO_USD.autopool_eth_addr, "fee_and_base_out"] = 0
limited_events_df.loc[limited_events_df["destination_in"] == AUTO_USD.autopool_eth_addr, "fee_and_base_in"] = 0
limited_events_df = limited_events_df.sort_values("block", ascending=True).dropna()
limited_events_df

,destination_in,destination_out,block,safe_value_out,rebalance_file_path,fee_and_base_out,fee_and_base_in,destination_in_name,destination_out_name,out_exchange_name,in_exchange_name,pool_in,pool_out
datetime,,,,,,,,,,,,,
2025-04-03 19:30:11+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xb6E89d23d1E33537D383622B1955EbDa135c0b5A,22190481,100.118752,rebalance_plan_1743708477_0xa7569A44f348d3D70d...,0.061408,0.000000,autoUSD,Staked USDe,none,tokemak,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497
2025-04-03 23:46:35+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x6F628dcCD275feA4277722D177265741031E09d7,22191759,99.992786,rebalance_plan_1743723845_0xa7569A44f348d3D70d...,0.005213,0.000000,autoUSD,Curve.fi Factory Plain Pool: crvUSD/USDT,curve,tokemak,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4
2025-04-04 15:57:35+00:00,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,22196597,25503.391475,rebalance_plan_1743782101_0xa7569A44f348d3D70d...,0.000000,0.023970,Curve.fi Strategic USD Reserves,autoUSD,tokemak,curve,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35
2025-04-04 17:11:35+00:00,0x7583b1589aDD33320366A48A92794D77763FAE9e,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,22196965,19127.543600,rebalance_plan_1743786510_0xa7569A44f348d3D70d...,0.000000,0.005733,Curve.fi Factory Plain Pool: crvUSD/USDT,autoUSD,tokemak,curve,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35
2025-04-04 18:58:59+00:00,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,22197502,14345.657699,rebalance_plan_1743792976_0xa7569A44f348d3D70d...,0.000000,0.023829,Curve.fi Strategic USD Reserves,autoUSD,tokemak,curve,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-21 06:17:35+00:00,0x7876F91BB22148345b3De16af9448081E9853830,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,24281313,219864.057000,rebalance_plan_1768976149_0xa7569A44f348d3D70d...,0.000000,0.034495,Fluid USD Coin,autoUSD,tokemak,fluid,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35
2026-01-21 08:17:35+00:00,0x7876F91BB22148345b3De16af9448081E9853830,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,24281913,111459.357500,rebalance_plan_1768983349_0xa7569A44f348d3D70d...,0.000000,0.034362,Fluid USD Coin,autoUSD,tokemak,fluid,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35
2026-01-22 04:17:35+00:00,0x7876F91BB22148345b3De16af9448081E9853830,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,24287885,654324.587500,rebalance_plan_1769055349_0xa7569A44f348d3D70d...,0.000000,0.034361,Fluid USD Coin,autoUSD,tokemak,fluid,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35


In [ ]:
break

In [3]:
import pandas as pd
from multicall import Call
import numpy as np

# Priority order: try getRate, then curve virtual price, then ERC4626-style convertToAssets(1e18)
VP_METHODS = [
    ("getRate", ["getRate()(uint256)"], None),
    ("get_virtual_price", ["get_virtual_price()(uint256)"], None),
    ("convertToAssets_1e18", ["convertToAssets(uint256)(uint256)", int(10**18)], int(10**18)),
]


def build_vp_calls(pool_address: str):
    calls = []
    for suffix, fn, _ in VP_METHODS:
        key = f"{pool_address}:{suffix}"
        calls.append(
            Call(
                target=pool_address,
                function=fn,
                returns=[(key, safe_normalize_with_bool_success)],
            )
        )
    return calls


def _coalesce_first_non_na(df: pd.DataFrame, cols_in_priority: list[str], out_col: str) -> pd.Series:
    cols = [c for c in cols_in_priority if c in df.columns]
    if not cols:
        df[out_col] = pd.NA
        return df[out_col]

    # First non-NA from left to right
    df[out_col] = df[cols].bfill(axis=1).iloc[:, 0]
    return df[out_col]


def compute_apr(vp_df):
    df = vp_df.copy()

    t0 = df.index[0]
    days = (df.index - t0).total_seconds() / 86400.0

    out0 = df["out_vp"].iloc[0]
    in0 = df["in_vp"].iloc[0]

    df["out_simple_pct"] = (df["out_vp"] / out0 - 1.0) * 100
    df["in_simple_pct"] = (df["in_vp"] / in0 - 1.0) * 100

    # annualized % using actual elapsed days; guard day=0 at start
    df["out_ann_pct"] = np.where(days > 0, ((df["out_vp"] / out0) ** (365.0 / days) - 1.0) * 100, np.nan)
    df["in_ann_pct"] = np.where(days > 0, ((df["in_vp"] / in0) ** (365.0 / days) - 1.0) * 100, np.nan)

    return df[["block", "out_vp", "in_vp", "out_simple_pct", "out_ann_pct", "in_simple_pct", "in_ann_pct"]]


from mainnet_launch.data_fetching.defi_llama.fetch_timestamp import fetch_blocks_by_unix_timestamps_defillama

today_block = ETH_CHAIN.get_block_near_top()

def determine_forward_looking_vp(row: pd.Series):
    start_block = int(row["block"])
    approx_blocks_per_day = 7150
    block_30_days = start_block + (approx_blocks_per_day * 30)
    block_60_days = start_block +( approx_blocks_per_day * 60)

    if (block_30_days > today_block) or (block_60_days  > today_block):
        return {
            **row,
            "actual_30_day_fee_and_base_out": None,
            "actual_60_day_fee_and_base_out": None,
            "actual_30_day_fee_and_base_in": None,
            "actual_60_day_fee_and_base_in": None,
            "start_block": start_block,
            "block_30_days": block_30_days,
            "block_60_days": block_60_days,
        }

    blocks_to_query = [start_block, block_30_days, block_60_days]

    out_addr = row["pool_out"]
    in_addr = row["pool_in"]

    calls_to_make = []
    calls_to_make += build_vp_calls(out_addr)
    calls_to_make += build_vp_calls(in_addr)

    vp_df = get_raw_state_by_blocks(
        calls_to_make,
        blocks_to_query,
        ETH_CHAIN,
        include_block_number=True,
    )

    # Coalesce per destination in the same priority order as VP_METHODS
    out_cols = [f"{out_addr}:{suffix}" for suffix, _, _ in VP_METHODS]
    in_cols = [f"{in_addr}:{suffix}" for suffix, _, _ in VP_METHODS]

    _coalesce_first_non_na(vp_df, out_cols, "out_vp")
    _coalesce_first_non_na(vp_df, in_cols, "in_vp")

    # Optional: keep only the useful columns
    keep = [c for c in ["block", "out_vp", "in_vp"] if c in vp_df.columns]
    apr_df = compute_apr(vp_df[keep].copy())

    actual_30_day_fee_and_base_out = apr_df.loc[apr_df["block"] == block_30_days, "out_ann_pct"].values[0] / 100
    actual_60_day_fee_and_base_out = apr_df.loc[apr_df["block"] == block_60_days, "out_ann_pct"].values[0] / 100

    actual_30_day_fee_and_base_in = apr_df.loc[apr_df["block"] == block_30_days, "in_ann_pct"].values[0] / 100
    actual_60_day_fee_and_base_in = apr_df.loc[apr_df["block"] == block_60_days, "in_ann_pct"].values[0] / 100

    if out_addr == AUTO_USD.autopool_eth_addr:
        actual_30_day_fee_and_base_out = 0.0
        actual_60_day_fee_and_base_out = 0.0
    if in_addr == AUTO_USD.autopool_eth_addr:
        actual_30_day_fee_and_base_in = 0.0
        actual_60_day_fee_and_base_in = 0.0

    return {
        **row,
        "actual_30_day_fee_and_base_out": actual_30_day_fee_and_base_out,
        "actual_60_day_fee_and_base_out": actual_60_day_fee_and_base_out,
        "actual_30_day_fee_and_base_in": actual_30_day_fee_and_base_in,
        "actual_60_day_fee_and_base_in": actual_60_day_fee_and_base_in,
        "start_block": start_block,
        "block_30_days": block_30_days,
        "block_60_days": block_60_days,
    }

In [7]:
import os
import pandas as pd
from tqdm import tqdm
import concurrent.futures as cf


def _safe_forward_vp(idx_row) -> pd.DataFrame:
    idx, row = idx_row
    try:
        out = determine_forward_looking_vp(row)  # DataFrame (e.g., 3 rows: start/30/60)

        # Attach identifiers so you can trace back to the source event
        out["event_idx"] = idx
        out["start_block"] = int(row["block"])
        if "tx_hash" in row:
            out["tx_hash"] = row["tx_hash"]
        if "destination_out" in row:
            out["destination_out"] = row["destination_out"]
        if "destination_in" in row:
            out["destination_in"] = row["destination_in"]

        out["failed"] = False
        out["error"] = None
        return out

    except Exception as e:
        # quick fail: return the original row as a 1-row DataFrame
        fail = row.to_frame().T
        fail["event_idx"] = idx
        fail["failed"] = True
        fail["error"] = repr(e)
        return fail

# limited_events_df = limited_events_df.head(5)  # limit for testing``
import time
import datetime

results = []
with cf.ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(_safe_forward_vp, idx_row): idx_row for idx_row in limited_events_df.iterrows()}
    
    for future in tqdm(cf.as_completed(futures), total=len(futures), desc="forward-looking VP"):
        results.append(future.result())

all_results_df = pd.DataFrame.from_records(results).set_index("event_idx")
timestamp_str = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"fee_and_base_apr_forward_looking_results_{timestamp_str}.csv"
all_results_df.to_csv(filename)
print(f"Saved results to {filename}")

forward-looking VP:   0%|          | 0/870 [00:00<?, ?it/s]/var/folders/7y/zqhjt5cx12n1p79c5pfxszvm0000gn/T/ipykernel_57596/82259458.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[out_col] = df[cols].bfill(axis=1).iloc[:, 0]
/var/folders/7y/zqhjt5cx12n1p79c5pfxszvm0000gn/T/ipykernel_57596/82259458.py:34: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[out_col] = df[cols].bfill(axis=1).iloc[:, 0]
forward-looking VP:   0%|          | 1/870 [00:00<07:57,  1.82it/s]/var/folders/7y/zqhjt5cx12n1p79c5pfxszvm0000gn/T/ipykernel_57596/82259458.py:34: Fut

Saved results to fee_and_base_apr_forward_looking_results_20260122_152053.csv


In [11]:
import plotly.express as px
import plotly.io as pio

pio.templates.default = None

px.scatter(
    all_results_df,
    x="fee_and_base_out",
    y="actual_30_day_fee_and_base_out",
    color="destination_out_name",
    title="Planned vs Actual 30-Day Fee+Base APR (Out)",
).add_trace(px.line(x=[0, 0.1], y=[0, 0.1]).data[0].update(line=dict(dash="dash", color="gray"), showlegend=False))

In [25]:
px.scatter(
    all_results_df,
    x="fee_and_base_in",
    y="actual_30_day_fee_and_base_in",
    color="destination_in_name",
    title="Expected vs Actual 30-Day Fee+Base APR (In)",
).add_trace(px.line(x=[0, 0.1], y=[0, 0.1]).data[0].update(line=dict(dash="dash", color="gray"), showlegend=False))

In [ ]:
# one limitation could be, we can only predict at the .1% level, no way we are accuarte closer than that

In [26]:
from plotly.subplots import make_subplots

import plotly.graph_objects as go

# Determine common x-axis range
x_min = min(all_results_df["actual_30_day_fee_and_base_in"].min(), all_results_df["fee_and_base_in"].min())
x_max = max(all_results_df["actual_30_day_fee_and_base_in"].max(), all_results_df["fee_and_base_in"].max())

# Create subplots with 2 rows and 1 column
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Distribution of actual fee + base APR we enter", 
                    "Distribution of expected fee + base APR we enter")
)

# Create histograms
fig1 = px.histogram(all_results_df, x="actual_30_day_fee_and_base_in")
fig2 = px.histogram(all_results_df, x="fee_and_base_in")

# Add traces
for trace in fig1.data:
    fig.add_trace(trace, row=1, col=1)

for trace in fig2.data:
    fig.add_trace(trace, row=2, col=1)

# Update x-axes to have the same range
fig.update_xaxes(range=[x_min, x_max], row=1, col=1)
fig.update_xaxes(range=[x_min, x_max], row=2, col=1)

# Update layout
fig.update_xaxes(title_text="Fee+Base APR", row=2, col=1)
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="Count", row=2, col=1)

fig.update_layout(height=700, showlegend=False)
fig.show()

In [21]:
plans

,file_name,datetime_generated,autopool_vault_address,chain_id,solver_address,rebalance_type,destination_out,token_out,destination_in,token_in,...,in_dest_apr,in_dest_adj_apr,apr_delta,swap_offset_period,num_candidate_destinations,candidate_destinations_rank,projected_swap_cost,projected_net_gain,projected_gross_gain,projected_slippage
0,rebalance_plan_1747876486_0xa7569A44f348d3D70d...,2025-05-22 01:14:46+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x7876F91BB22148345b3De16af9448081E9853830,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,...,0.054361,0.054323,0.054323,14.0,0.0,NaN,48.696771,0.000000,0.000000,0.015000
1,rebalance_plan_1751790625_0xa7569A44f348d3D70d...,2025-07-06 08:30:25+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rebalance_plan_1743982165_0xa7569A44f348d3D70d...,2025-04-06 23:29:25+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rebalance_plan_1744695741_0xa7569A44f348d3D70d...,2025-04-15 05:42:21+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x7583b1589aDD33320366A48A92794D77763FAE9e,0x390f3595bCa2Df7d23783dFd126427CCeb997BF4,...,0.073199,0.072660,0.072660,12.0,3.0,0.0,25.234559,138.624499,163.859058,0.032486
4,rebalance_plan_1745052157_0xa7569A44f348d3D70d...,2025-04-19 08:42:37+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xE4545f9dBC30Ccb6Cda6930DDFd69f3D419FcB61,0x5C20B550819128074FD538Edf79791733ccEdd18,...,0.064439,0.064409,0.064409,12.0,3.0,0.0,23.367432,260.629399,283.996831,0.015578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1690,rebalance_plan_1765700143_0xa7569A44f348d3D70d...,2025-12-14 08:15:43+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x366C094C5563CD12AF27b9AfFF2200B0E0D056E0,0x85B2b559bC2D21104C4DEFdd6EFcA8A20343361D,...,0.103896,0.103697,0.103697,15.0,3.0,0.0,6.019116,228.556255,234.575371,0.010329
1691,rebalance_plan_1756880141_0xa7569A44f348d3D70d...,2025-09-03 06:15:41+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x7876F91BB22148345b3De16af9448081E9853830,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,...,0.082677,0.082676,0.082676,15.0,3.0,0.0,3.000000,58.699346,61.699346,0.015000
1692,rebalance_plan_1769062549_0xa7569A44f348d3D70d...,2026-01-22 06:15:49+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x7876F91BB22148345b3De16af9448081E9853830,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,...,0.043441,0.043431,0.043431,15.0,0.0,NaN,49.074345,0.000000,0.000000,0.015000
1693,rebalance_plan_1769055349_0xa7569A44f348d3D70d...,2026-01-22 04:15:49+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,1,0xD02b50CFc6c2903bF13638B28D081ad11515B6f9,FromIdle,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x7876F91BB22148345b3De16af9448081E9853830,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,...,0.043361,0.043343,0.043343,15.0,0.0,NaN,98.148689,0.000000,0.000000,0.015000


In [18]:
px.ecdf(
    all_results_df.dropna(subset=["actual_30_day_fee_and_base_in"]),
    x=["actual_30_day_fee_and_base_in", "actual_60_day_fee_and_base_in", "fee_and_base_in"],
    title="ECDF of actual fee + base APR we enter",
).show()

In [20]:
all_results_df

,destination_in,destination_out,block,safe_value_out,rebalance_file_path,fee_and_base_out,fee_and_base_in,destination_in_name,destination_out_name,out_exchange_name,...,pool_out,actual_30_day_fee_and_base_out,actual_60_day_fee_and_base_out,actual_30_day_fee_and_base_in,actual_60_day_fee_and_base_in,start_block,block_30_days,block_60_days,failed,error
event_idx,,,,,,,,,,,,,,,,,,,,,
2025-04-08 17:02:47+00:00,0xd7900d87069C815a299bdA7aFDcd7eEe98fe4b6c,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,22225549,64072.851183,rebalance_plan_1744131598_0xa7569A44f348d3D70d...,0.020147,0.050253,Curve Strategic Ethena Reserves,Curve.fi Strategic USD Reserves,curve,...,0x4f493B7dE8aAC7d55F71853688b1F7C8F0243C85,0.011073,0.010168,0.011101,0.024830,22225549,22440049,22654549,False,None
2025-04-05 04:07:11+00:00,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,22200224,24208.297400,rebalance_plan_1743825875_0xa7569A44f348d3D70d...,0.000000,0.023809,Curve.fi Strategic USD Reserves,autoUSD,tokemak,...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0.000000,0.000000,0.010921,0.010562,22200224,22414724,22629224,False,None
2025-04-04 15:57:35+00:00,0xbB2d2dd491204a86ec10a1a6972F940B34fE060e,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,22196597,25503.391475,rebalance_plan_1743782101_0xa7569A44f348d3D70d...,0.000000,0.023970,Curve.fi Strategic USD Reserves,autoUSD,tokemak,...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0.000000,0.000000,0.010747,0.010576,22196597,22411097,22625597,False,None
2025-04-04 17:11:35+00:00,0x7583b1589aDD33320366A48A92794D77763FAE9e,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,22196965,19127.543600,rebalance_plan_1743786510_0xa7569A44f348d3D70d...,0.000000,0.005733,Curve.fi Factory Plain Pool: crvUSD/USDT,autoUSD,tokemak,...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0.000000,0.000000,0.004295,0.004193,22196965,22411465,22625965,False,None
2025-04-03 19:30:11+00:00,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0xb6E89d23d1E33537D383622B1955EbDa135c0b5A,22190481,100.118752,rebalance_plan_1743708477_0xa7569A44f348d3D70d...,0.061408,0.000000,autoUSD,Staked USDe,none,...,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497,0.044399,0.050761,0.000000,0.000000,22190481,22404981,22619481,False,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-17 08:17:47+00:00,0x7583b1589aDD33320366A48A92794D77763FAE9e,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,23817651,20000.000000,rebalance_plan_1763367343_0xa7569A44f348d3D70d...,0.000000,0.014130,Curve.fi Factory Plain Pool: crvUSD/USDT,autoUSD,tokemak,...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0.000000,0.000000,0.009763,0.006502,23817651,24032151,24246651,False,None
2025-11-17 06:17:47+00:00,0x65efCF2cce562DCBf07e805eEbeDeF21Dbd8Ea3D,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,23817054,10565.060000,rebalance_plan_1763360143_0xa7569A44f348d3D70d...,0.000000,0.009552,Curve.fi Factory Plain Pool: crvUSD/USDC,autoUSD,tokemak,...,0xa7569A44f348d3D70d8ad5889e50F78E33d80D35,0.000000,0.000000,0.007065,0.005006,23817054,24031554,24246054,False,None
2025-11-23 06:17:47+00:00,0x8acA8accFb69adeff607431E0F25466B7b76a8Ad,0x7876F91BB22148345b3De16af9448081E9853830,23859751,39997.959321,rebalance_plan_1763878543_0xa7569A44f348d3D70d...,0.037689,0.105932,Savings crvUSD,Fluid USD Coin,fluid,...,0x9Fb7b4477576Fe5B32be4C1843aFB1e55F251B33,0.037632,0.035688,0.033805,0.023613,23859751,24074251,24288751,False,None


In [ ]:
# clip to 5%, prevent the worst outliers

In [22]:
from plotly.subplots import make_subplots

import plotly.graph_objects as go

# Create subplots with 1 row and 2 columns
fig = make_subplots(rows=1, cols=2, subplot_titles=("Expected Fee+Base APR (In)", "Actual 30-Day Fee+Base APR (In)"))

# ECDF for expected (fee_and_base_in)
plot_df_expected = all_results_df.dropna(subset=["fee_and_base_in"]).copy()
fig_expected = px.ecdf(
    plot_df_expected,
    x="fee_and_base_in",
)
for trace in fig_expected.data:
    fig.add_trace(trace, row=1, col=1)

# ECDF for actual 30-day (actual_30_day_fee_and_base_in)
plot_df_actual = all_results_df.dropna(subset=["actual_30_day_fee_and_base_in"]).copy()
fig_actual = px.ecdf(plot_df_actual, x="actual_30_day_fee_and_base_in")
for trace in fig_actual.data:
    fig.add_trace(trace, row=1, col=2)

# Update layout
fig.update_xaxes(title_text="Fee+Base APR", row=1, col=1)
fig.update_xaxes(title_text="Fee+Base APR", row=1, col=2)
fig.update_yaxes(title_text="Cumulative Probability", row=1, col=1)
fig.update_yaxes(title_text="Cumulative Probability", row=1, col=2)

fig.update_layout(title_text="Expected vs Actual Fee+Base APR Distribution (In)", height=500, showlegend=True)

fig.show()

In [ ]:
break

In [ ]:
all_results_df["30_day_out_diff"] = (
    all_results_df["actual_30_day_fee_and_base_out"] - all_results_df["fee_and_base_out"]
)
all_results_df["60_day_out_diff"] = (
    all_results_df["actual_60_day_fee_and_base_out"] - all_results_df["fee_and_base_out"]
)
all_results_df["30_day_in_diff"] = all_results_df["actual_30_day_fee_and_base_in"] - all_results_df["fee_and_base_in"]
all_results_df["60_day_in_diff"] = all_results_df["actual_60_day_fee_and_base_in"] - all_results_df["fee_and_base_in"]


import plotly.express as px

plot_df = all_results_df.dropna(subset=["destination_out_name", "30_day_out_diff"]).copy()

fig = px.ecdf(
    plot_df,
    x="30_day_out_diff",
    color="destination_out_name",  # one ECDF line per destination
    title="Difference between Actual and Planned 30-Day Fee+Base APR (Out)",
)

fig.update_layout(
    legend_title_text="Destination (out)",
)
print("diff >0 means actual > expected")
print("diff <0 means actual < expected")
fig.show()

In [ ]:
plot_df = all_results_df.dropna(subset=["destination_in_name", "30_day_in_diff"]).copy()

fig = px.ecdf(
    plot_df,
    x="30_day_in_diff",
    color="destination_in_name",
    title="Difference between Actual and Planned 30-Day Fee+Base APR (In)",
)

fig.update_layout(
    legend_title_text="Destination (in)",
)
print("diff >0 means actual > expected")
print("diff <0 means actual < expected")
fig.show()

In [ ]:
# maybe we should be doing the lowest hanging fruit first? like the ones that are way off?
# Calculate absolute differences for sorting
all_results_df["abs_30_day_out_diff"] = all_results_df["30_day_out_diff"].abs()
all_results_df["abs_60_day_out_diff"] = all_results_df["60_day_out_diff"].abs()
all_results_df["abs_30_day_in_diff"] = all_results_df["30_day_in_diff"].abs()
all_results_df["abs_60_day_in_diff"] = all_results_df["60_day_in_diff"].abs()

# Find the most off predictions for each category
print("=" * 80)
print("TOP 10 WORST PREDICTIONS (60-day OUT)")
print("=" * 80)
all_results_df.columns

In [ ]:
interesting_cols = [
    "destination_in_name",
    "destination_out_name",
    "actual_30_day_fee_and_base_out",
    "actual_60_day_fee_and_base_out",
    "actual_30_day_fee_and_base_in",
    "actual_60_day_fee_and_base_in",
    "fee_and_base_out",
    "fee_and_base_in",
    "30_day_out_diff",
    "60_day_out_diff",
    "30_day_in_diff",
    "60_day_in_diff",
    "abs_30_day_out_diff",
    "block",
    "safe_value_out",
]


all_results_df.sort_values(by="abs_60_day_out_diff", ascending=False)[interesting_cols].head(10)